In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

import numpy as np
import pandas as pd
import time
import re 

from IPython.display import Image

#Word cloud
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image # 만약 "No module named 'PIL'" 에러가 발생하면 [ pip install Pillow==5.4.1 ] 로 라이브러리를 설치해줍니다.
from nltk.corpus import stopwords


# 유사도 분석
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#텍스트 분석
# from konlpy.tag import Okt
# okt = Okt()

# 1. Preprocessing 

## 1-1)  SongID 크롤링 

Chrome Driver path 만 변경하면 됨!

In [2]:
## 멜론 사이트에서 각 장르별 스테디셀러 탑500곡을 얻기위해, 4000곡에 대한 SongID 크롤링 ##

# 장르명칭 및 장르코드
list_gnrname = ['발라드','댄스','랩/힙합','R&B/Soul','인디음악','록/메탈','트로트','포크/블루스']
list_gnrcode = ['GN0100','GN0200','GN0300','GN0400','GN0500','GN0600','GN0700','GN0800']

# 크롬 드라이버
driver = webdriver.Chrome(executable_path='/opt/homebrew/bin/chromedriver') # chrome driver path 만 설정!!
# driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')

# 전체 장르 songid 리스트
total_song_id = []

# 장르별 스테디셀러 songid 추출 for문 (8개 장르)
for gnrcode in list_gnrcode[:8]:
    song_id = []

    # url의 startindex 변경 -> Page 변경 (10페이지, 장르당 500곡)
    for start_idx in range(1,452,50):
        songlist_url = 'https://www.melon.com/genre/song_list.htm?gnrCode='+ gnrcode \
        + '&dtlGnrCode=#params[gnrCode]=' + gnrcode \
        + '&params[dtlGnrCode]=&params[orderBy]=POP&params[steadyYn]=N&po=pageObj' \
        + '&startIndex=' + str(start_idx)

        driver.get(songlist_url)
        time.sleep(1)
        web = driver.page_source
        source = BeautifulSoup(web,'html.parser')
        
        # 노래 고유번호인 Songid를 추출
        for song_info in source.find_all('div',{'class':'ellipsis rank01'}):
            print(gnrcode, '크롤링 중입니다 : ', song_info.find('a')['title'][:-2])
            id = song_info.find('a')['href']
            id = re.findall("\d+", id)[-1]  
            song_id.append(id)
        print('****다음 페이지로 넘어갑니다****')
    
    # 전체 song_id 리스트에 append
    total_song_id.append(song_id)
    
    # 하나의 장르 크롤링 완료
    print("---------------------------------------------------")
    print("새로운 장르로 넘어갑니다")

print("---------------------------------------------------")
print("8개 장르, 스테디셀러 4000곡의 크롤링을 성공적으로 마쳤습니다!!")

driver.close()
driver.quit()

# 장르별 songid DataFrame
df_song_id = pd.DataFrame(total_song_id)
df_song_id = np.transpose(df_song_id)
df_song_id.columns = list_gnrname # Column을 장르명으로 변경

# 파일 저장
file_song_id = 'SongId_8genre_4000songs.xlsx'
df_song_id.to_excel(file_song_id, index=False, encoding='utf-8')

/var/folders/rz/3h16f8bd5s36k1xv36vgkl1m0000gn/T/ipykernel_35668/95016978.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/opt/homebrew/bin/chromedriver') # chrome driver path 만 설정!!


KeyboardInterrupt: 

In [4]:
df_song_id.tail()

,발라드,댄스,랩/힙합,R&B/Soul,인디음악,록/메탈,트로트,포크/블루스
495,1902386,5785424,4330746,None,None,4682457,None,None
496,7872705,3956314,4011597,None,None,3981992,None,None
497,4152209,2218824,None,None,None,2669855,None,None
498,None,1765763,None,None,None,1762073,None,None
499,None,2688949,None,None,None,2192648,None,None


### 1-2) 얻은 Song ID로 제목/가수/발매일/장르/가사 scraping

#### Test sample

In [7]:
##Testing web scraping with sample data
# Sample data: 아이유-스물셋 - 
songid = 7896508
url = "https://www.melon.com/song/detail.htm?songId="+ str(songid)

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

web = requests.get(url,headers=headers).content
web_page = BeautifulSoup(web, 'html.parser')

In [8]:
# 제목
title = web_page.find('div',{'class':'song_name'}).get_text().strip()
title = title.split('\t')[-1]
# 가수명
artist = web_page.find('a',{'class':'artist_name'})['title']
# 발매일, 장르
date_genre = web_page.find('dl',{'class':'list'}).get_text()
date = date_genre.split('\n')[4]
genre = date_genre.split('\n')[6]
#가사
lyric = web_page.find('div',{'class':'lyric'}).get_text().strip()

print(title, artist, date, genre)
print()
print(lyric)

스물셋 아이유 2015.10.23 댄스

I'm twenty three 난 수수께끼 Question뭐게요 맞혀봐요I'm twenty three틀리지 말기 Because 난 몹시 예민해요맞혀봐한 떨기 스물셋 좀 아가씨 태가 나네다 큰 척해도 적당히 믿어줘요얄미운 스물셋아직 한참 멀었다 얘 덜 자란 척해도대충 속아줘요난 그래 확실히 지금이 좋아요 아냐 아냐사실은 때려 치고 싶어요아 알겠어요나는 사랑이 하고 싶어아니 돈이나 많이 벌래맞혀봐 어느 쪽이게얼굴만 보면 몰라속마음과 다른 표정을 짓는 일아주 간단하거든어느 쪽이게사실은 나도 몰라애초에 나는 단 한 줄의 거짓말도 쓴 적이 없거든여우인 척 하는 곰인 척하는 여우 아니면아예 다른 거어느 쪽이게뭐든 한 쪽을 골라색안경 안에 비춰지는 거뭐 이제 익숙하거든Check it out겁나는 게 없어요엉망으로 굴어도 사람들은 내게 매일 친절해요인사하는 저 여자모퉁이를 돌고도아직 웃고 있을까늘 불안해요난 영원히 아이로 남고 싶어요아니 아니물기 있는 여자가 될래요아 정했어요난 죽은 듯이 살래요아냐 다 뒤집어 볼래맞혀봐어느 쪽이게얼굴만 보면 몰라속마음과 다른 표정을 짓는 일아주 간단하거든어느 쪽이게사실은 나도 몰라애초에 나는 단 한 줄의 거짓말도 쓴 적이 없거든여우인 척 하는 곰인 척하는 여우 아니면아예 다른 거어느 쪽이게뭐든 한 쪽을 골라색안경 안에 비춰지는 거뭐 이제 익숙하거든난 당신 맘에 들고 싶어요아주 살짝만얄밉게 해도 돼요난 당신 맘에 들고 싶어요자기 머리 꼭대기 위에서놀아도 돼요맞혀봐어느 쪽이게얼굴만 보면 몰라속마음과 다른 표정을 짓는 일아주 간단하거든어느 쪽이게사실은 나도 몰라애초에 나는 단 한 줄의 거짓말도 쓴 적이 없거든여우인 척 하는 곰인 척하는 여우 아니면아예 다른 거어느 쪽이게뭐든 한 쪽을 골라색안경 안에 비춰지는 거뭐 이제 익숙하거든


#### 스테디셀러 4000곡 정보 가공

In [9]:
# 장르별 songid 파일 
df_genre_songid = pd.read_excel('SongId_8genre_4000songs.xlsx') 
df_genre_songid.head() 

,발라드,댄스,랩/힙합,R&B/Soul,인디음악,록/메탈,트로트,포크/블루스
0,3414749.0,3053259,3853980.0,5523623.0,4264561.0,3753304,2710613.0,4264561.0
1,4383833.0,7896508,5650996.0,19807.0,5466962.0,3620493,844269.0,5466962.0
2,1944399.0,3051244,1850253.0,1139741.0,3620493.0,1833966,3896698.0,5605860.0
3,4579468.0,1325050,531840.0,3910821.0,4451155.0,4451155,5716847.0,5700639.0
4,4475939.0,3625504,2312253.0,519620.0,5694902.0,837567,844268.0,4583482.0


### 1-3) Web scraping 프로세스 함수화: genre_songid_to_info 함수
#### 해당장르에 속하는 곡들의 곡정보를 '장르명_500songs.xlsx'로 저장 -> 8개 탑500곡 장르 파일 저장 완료!

### 8개 장르마다 Dataframe 생성하는 함수: songid_to_info 함수
#### Songid들이 들어있는 '리스트'를 넣어주면 곡정보 dataframe으로 변환

In [25]:
def songid_to_info(songlist):
    #변수 담을 리스트들 생성
    titles = []
    artists = []
    dates = []
    genres = []
    lyrics = []

    #곡 정보 받아오기
    idx=0
    for songid in songlist[:10]:
        url = "https://www.melon.com/song/detail.htm?songId="+ str(songid)

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(url,headers=headers).content
        web_page = BeautifulSoup(web, 'html.parser')

        #노래 제목
        title = web_page.find('div',{'class':'song_name'}).get_text().strip().split('\t')[-1]
        #title = title.split('(')[0].strip() # (feat.) 등 제거
        titles.append(title)
        #가수명
        artist = web_page.find('a',{'class':'artist_name'})['title']
        artists.append(artist)
        #곡 발매일, 장르
        date_genre = web_page.find('dl',{'class':'list'}).get_text()
        date = date_genre.split('\n')[4]
        genre = date_genre.split('\n')[6]
        dates.append(date)
        genres.append(genre)
        #가사
        try: 
            lyric = web_page.find('div',{'class':'lyric'}).get_text().strip()
            lyrics.append(lyric)
        except: # 성인인증이 필요한 경우 스크래핑 불가 -> 가사 생략
            print('----{} : 성인인증이 필요한 가사입니다----'.format(title))
            lyrics.append('.')

        idx += 1 #다음곡으로 넘어가기
        print(idx, '번째곡 크롤링 완료 :', title)

    info_dict = {'Title':titles, 
                 'Artist':artists, 
                 'Date':dates, 
                 'Genre':genres, 
                 'Lyric':lyrics}

    df_song_info = pd.DataFrame(info_dict)
    return df_song_info

In [34]:
df_댄스 = songid_to_info(df_genre_songid['댄스'])
df_댄스.tail()

1 번째곡 크롤링 완료 : 미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ)
2 번째곡 크롤링 완료 : 스물셋
3 번째곡 크롤링 완료 : 좋은 날
4 번째곡 크롤링 완료 : 비행기
5 번째곡 크롤링 완료 : 너랑 나
6 번째곡 크롤링 완료 : 하루 끝
7 번째곡 크롤링 완료 : 거짓말 (Remix)
8 번째곡 크롤링 완료 : SHAKE IT
9 번째곡 크롤링 완료 : 오늘부터 우리는 (Me Gustas Tu)
10 번째곡 크롤링 완료 : I Don't Care


,Title,Artist,Date,Genre,Lyric
5,하루 끝,아이유,2012.05.11,댄스,Monday Better day처음처럼 설레이는 그런 날Sunday Better d...
6,거짓말 (Remix),BIGBANG (빅뱅),2008.11.05,댄스,거짓말늦은 밤 비가 내려와 널 데려와젖은 기억 끝에 뒤척여 나너 없이 잘 살 수 있...
7,SHAKE IT,씨스타,2015.06.22,댄스,너만을 유혹하는 춤 우우우 우우우우심장에 매력 발산 중 우우우 우우우우손끝만 스쳐도...
8,오늘부터 우리는 (Me Gustas Tu),여자친구 (GFRIEND),2015.07.23,댄스,널 향한 설레임을오늘부터 우리는 꿈꾸며 기도하는오늘부터 우리는저 바람에 노을 빛내 ...
9,I Don't Care,2NE1,2009.07.08,댄스,hey playboyit's about time and your time's upI...


In [29]:
def genre_songid_to_info(genre_name): 
    start = time.time()

    print('-------------------------------------------------------------------------')
    print('[{}] 장르 크롤링 시작합니다'.format(genre_name))
    
    songlist = df_genre_songid[genre_name].tolist()
    df_genre_songid_to_info = songid_to_info(songlist)
    
    filename = "test_500songs.xlsx".format(genre_name.split('/')[0]) 
    df_genre_songid_to_info.to_excel(filename, index=False, encoding='utf-8')

    print("크롤링 완료!! 엑셀 파일을 확인하세요!")

    print()
    print(round((time.time()-start)/60,2), '분 소요')
    return df_genre_songid_to_info.head()



### 1-4) 장르별로 Top 500곡 Title/Artist/Date/Genre/Lyric: Web Crawling  

In [33]:
list_gnrname = ['발라드','댄스','랩/힙합','R&B/Soul','인디음악','록/메탈','트로트','포크/블루스']
#발라드 크롤링
gnr_selected = list_gnrname[2] # 정보 원하는 장르 선택

genre_songid_to_info(gnr_selected)

-------------------------------------------------------------------------
[랩/힙합] 장르 크롤링 시작합니다


AttributeError: 'NoneType' object has no attribute 'get_text'

In [11]:
# 모든 8개 장르별로 크롤링 
for i in range(len(list_gnrname)):
    gnr_selected = list_gnrname[i] # 정보 원하는 장르 선택
    genre_songid_to_info(gnr_selected)

-------------------------------------------------------------------------
[발라드] 장르 크롤링 시작합니다


AttributeError: 'NoneType' object has no attribute 'get_text'